In [1]:
from numpy import linalg as LA
import matplotlib.pyplot as plt
import sys
import os, shutil                         # For issuing commands to the OS.
from os import listdir
from os.path import isfile, join
import time
import random
from matplotlib.font_manager import FontProperties
import pandas as pd
import csv
from scipy import stats
import matplotlib.pyplot as plt
import datetime as dt
import datetime
from scipy.optimize import curve_fit
from scipy.stats import bernoulli
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import PCA
import numpy as np
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
#from sklearn.metrics import confusion_matrixfrom sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import brier_score_loss
from sklearn.preprocessing import PolynomialFeatures
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from io import BytesIO
import boto3
import urllib
import netCDF4 as nc
from netCDF4 import MFDataset
import urllib.request
from time import sleep
import requests
from netCDF4 import Dataset
import timeit
#-------------------------------------------------------------------------------------------------------------
# VO(t, p, latitude, longitude) (s**-1) 
# T(t, p, latitude, longitude)   (K)
# W(t, p, latitude, longitude)   (Pa s**-1)
# R(t, p, latitude, longitude)   (%)
# CC(t, p, latitude, longitude)  (0 - 1)

# ggap:
# t, latitude = 256, longitude = 512
# p = 37
# p = 1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 700, 650, 600, 550, 500, 450, 400, 350,300, 250, 
#     225, 200, 175, 150, 125, 100, 70, 50,30, 20, 10, 7, 5, 3, 2, 1 ;
#-------------------------------------------------------------------------------------------------------------
# there are 37 pressure height levels
# select pressure height and latitude levels
pr0 =  0 #7 # 400mb
pr1 =  25 # 15

lat0 = 50 # 10 #150  # 5 # input in degrees N, must be more southward than lat1
lat1 = 60 #60 #200# 60  # 90

start = timeit.default_timer()
fps = 3
# directories and create lists of years in strings to be able to read in the files, which contain year in name. 
gafs = 'gafs01051979/gafs197901050003.nc' # 00/12 + 3 6 9 12
ggap = 'ggap01051979/ggap197901050000-c.nc' # 00, 06, 12, 18
dir = '/Volumes/Seagate Backup Plus Drive/meteo-badc/ggap/' # 1979-1993 (1995-2003)
dir1 = '/Volumes/Seagate Backup Plus Drive/meteo-2/ggap/' # 1994-2018
#-------------------------------------------------------------------------------------------------------------
# 1979-1993
year0 = 1979
yrs = [(str(year0+i)+'/') for i in range(15)]
yrs_ = [(year0+i) for i in range(15)]
#print(yrs1)
# 1994-2018, in combination with a different hard disk and path
yrs1 = [(str(year0+i)+'/') for i in range(15,40)]
yrs1_ = [(year0+i) for i in range(15,40)]
#print(yrs1)
# choose the hard disk (the data are distributed over 2 disks)
di = 1
if di == 0:
    dir = dir
    yrs = yrs
    yrs_ = yrs_
    mypath = [dir + ye for ye in yrs]
else:
    dir = dir1
    yrs = yrs1
    yrs_ = yrs1_
    mypath = [dir1 + ye for ye in yrs1]    

In [2]:
# This function lists files from path    
def name_is_good(f):
    if f[-3:] == '.nc':
        return True
    else:
        False
        
def files_per_day(dir,yr):   # yr is a single year 
    mypaths = dir + yr
    files_in_year = [f for f in listdir(mypaths) if isfile(join(mypaths, f)) and name_is_good(f)]     
    #dates2 = [dates[d] for d in range(len(dates)) if d % 5 == 0] # take only every 5th day, for plots
    #d = dates[day] # has 365 inputs because of np.unique only takes sorted unique value...same day :-)
    dates = np.unique(np.array([f[4:12] for f in files_in_year])) # indices 4:12 in the filenames ggapYYYYDDHHHH..nc    
    return dates,files_in_year # days in one year

def Read_grid(mypath,files_for_day):
    f_all = []
    for file in files_for_day:  # daily values
        f = Dataset(mypath + file, 'r')
        #print(mypath + file)
        f_all.append(f)          
        p = f.variables['p'][:]
        lat = f.variables['latitude'][:] # lat = [90,...0,...,-90]
        #print(lat)
        lon = f.variables['longitude'][:]
    [f.close() for f in f_all]    
    return lat,lon,p


def Read_files(mypath,files_for_day,var):
    total = []                            # (t,p,lat,lon)
    f_all = []
    for file in files_for_day:            # daily values
        #print(mypath + file)
        f = Dataset(mypath + file, 'r')
        f_all.append(f)            
        w = f.variables[var] 
        total.append(w)      
    total = np.asarray(total)
    total = np.sum(total, axis=0) / len(files_for_day) # calculate daily mean values
    [f.close() for f in f_all]    
    return total


def lat_to_grid(latNorth,lat):
        # latNorth is the latitude North counted from the equator, whereas the files start counting coordinates 
        # from the Nort pole
        grid_length_in_degrees = 180/float(len(lat)) # nb of degrees per latitude grid coordinate 
        grid_coordinate = int((90-latNorth)/float(grid_length_in_degrees))# given the latitude in degrees N, what is the 
        return grid_coordinate                                     # coordinate index of the given nc.file?

def Main(files_for_day,mypath,p0, p1, lat0, lat1):
    
        lat,lon,p = Read_grid(mypath,files_for_day)
        T_tot = Read_files(mypath,files_for_day,'T')
        T_tot = T_tot -273.15    # conversion to celsius
        
        l0 = lat_to_grid(lat0,lat) # 50N
        l1 = lat_to_grid(lat1,lat) # 60N
        #print(lat)
        #print('lat0,1    ',lat0,lat1)
        #print('gridlat0,1',l0,l1)
        #print('test lat0,1',lat[l0],lat[l1])
        T_tota = T_tot[0,p0:p1,l1:l0,0:len(lon)] 
        T_tota = np.flip(T_tota, axis=1) 
        T_total = np.mean(T_tota, axis = 2)        # mean temperature along longitudes
        #
        
        #vo_total = Read_files(mypath,files_for_day,'VO')
        #W_total = Read_files(mypath,files_for_day,'W')
        #CC_total = Read_files(mypath,files_for_day,'CC')
        
        #v_ = v[0,p0:p1,l0:l1,0:len(lon)]
        #w_ = w[0,p0:p1,l0:l1,0:len(lon)]
        
        return T_total    

In [ ]:
#T_t = Main(files_for_day,mypaths,pr0, pr1, lat0, lat1)


In [3]:
#for k in range(len(yrs_)): # for each year

import time
start_time = time.time()
print('lats: ',lat0,lat1)
print('pr: ',pr0,pr1)
for k in range(len(yrs_)):  
    mypaths = dir + yrs[k]
    dates,files_in_year = files_per_day(dir,yrs[k]) # days in one year
    print('year ', yrs_[k])
    timestep = 0    
    T_ = []           
    file_name = 'Tno_'+str(yrs_[k])+'__.txt'
    f = open(file_name,'w+')

    for d in dates:  # all files, sorted according to the date, for a given year. Each file corresponds to a year        
        files_for_day = [f for f in files_in_year if f[4:12] == d]  # list of files corresponding to the given day d in the loop       
        lat,lon,p = Read_grid(mypaths,files_for_day)          
        T_tot = Main(files_for_day,mypaths,pr0, pr1, lat0, lat1)
        #print('T for day',d, T_tot[:,:])
        for line in T_tot[:,:]:    
            #print('T ',(T_tot[x,:] for x in line))
            f.write(' '.join(str(x) for x in line)+'\n')           
        #f.write('# New slice\n') 
        T_.append(T_tot[:,:])
        timestep = timestep + 1       
    f.close()
    
    print('year ', yrs_[k], 'mean T (C)', np.mean(T_))
    print("--- %s seconds ---" % (time.time() - start_time))

lats:  50 60
pr:  0 25
year  1994
year  1994 mean T (C) -21.195375
--- 982.1127760410309 seconds ---
year  1995
year  1995 mean T (C) -20.959826
--- 1973.497666835785 seconds ---
year  1996
year  1996 mean T (C) -21.396648
--- 2965.8446350097656 seconds ---
year  1997
year  1997 mean T (C) -20.981768
--- 3957.4904437065125 seconds ---
year  1998
year  1998 mean T (C) -20.844543
--- 4950.238115787506 seconds ---
year  1999
year  1999 mean T (C) -20.88337
--- 5935.653341054916 seconds ---
year  2000
year  2000 mean T (C) -21.090584
--- 6907.562319993973 seconds ---
year  2001
year  2001 mean T (C) -20.85878
--- 7862.906587839127 seconds ---
year  2002
year  2002 mean T (C) -20.876959
--- 8824.290944099426 seconds ---
year  2003
year  2003 mean T (C) -20.641989
--- 9785.895998954773 seconds ---
year  2004
year  2004 mean T (C) -20.730814
--- 10758.357133865356 seconds ---
year  2005
year  2005 mean T (C) -20.509863
--- 39017.33997178078 seconds ---
year  2006
year  2006 mean T (C) -20.644

In [3]:
    
    #with open(file_name,'w') as f: # (4,5,10)
        #f.write('# Array shape: {0}\n'.format(data.shape))
        #for data_slice in f:
        #   np.savetxt(f, data_slice, fmt='%-7.2f')
        #   f.write('# New slice\n')
    

data = np.arange(200).reshape((4,5,10))

# Write the array to disk
with open('test.txt', 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(data.shape))

    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for data_slice in data:

        # The formatting string indicates that I'm writing out
        # the values in left-justified columns 7 characters in width
        # with 2 decimal places.  
        np.savetxt(outfile, data_slice, fmt='%-7.2f')

        # Writing out a break to indicate different slices...
        outfile.write('# New slice\n')

IndentationError: expected an indented block (<ipython-input-3-88bc2aed69a2>, line 15)